# 1. Packages

First, I imported all packages that are needed to be used.

In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from util import *

%matplotlib inline

# 2. Data

Here, I loaded the dataset.

In [2]:
X, y = load_data("data/processed/dataset_1.csv")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Then, I checked the first few instances of the dataset.

In [4]:
X_train.head(5)

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
68,9.029,17.33,58.79,250.5,0.10660,0.14130,0.31300,0.04375,0.2111,0.08046,...,10.31,22.65,65.50,324.7,0.14820,0.43650,1.25200,0.17500,0.4228,0.11750
181,21.090,26.57,142.70,1311.0,0.11410,0.28320,0.24870,0.14960,0.2395,0.07398,...,26.68,33.48,176.50,2089.0,0.14910,0.75840,0.67800,0.29030,0.4098,0.12840
63,9.173,13.86,59.20,260.9,0.07721,0.08751,0.05988,0.02180,0.2341,0.06963,...,10.01,19.23,65.59,310.1,0.09836,0.16780,0.13970,0.05087,0.3282,0.08490
248,10.650,25.22,68.01,347.0,0.09657,0.07234,0.02379,0.01615,0.1897,0.06329,...,12.25,35.19,77.98,455.7,0.14990,0.13980,0.11250,0.06136,0.3409,0.08147
60,10.170,14.88,64.55,311.9,0.11340,0.08061,0.01084,0.01290,0.2743,0.06960,...,11.02,17.45,69.86,368.6,0.12750,0.09866,0.02168,0.02579,0.3557,0.08020


In [5]:
y_train.head(5)

68     0
181    1
63     0
248    0
60     0
Name: class, dtype: int64

After that, I checked the dimensions of the data.

In [6]:
print("The shape of X_train is: ", X_train.shape)
print("The shape of y_train is: ", y_train.shape)

The shape of X_train is:  (455, 30)
The shape of y_train is:  (455,)


Finally, I convert them to numpy for easier calculation.

In [9]:
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()
X_test = X_test.to_numpy()
y_test = y_test.to_numpy()

# 3. Model implementation

## 3.1. Sigmoid function

In logistic regression, the model is presented as:
$$ f_{\mathbf{w},b}(x) = g(\mathbf{w}\cdot \mathbf{x} + b)$$
Where function $g$ is the sigmoid function. The formula for the sigmoid function is:
$$g(z) = \frac{1}{1+e^{-z}}$$

In this part, I implemented the sigmoid function.

In [10]:
def sigmoid(z):
    """
    Compute the sigmoid of z

    Args:
        z (ndarray): A numpy array.

    Returns:
        g (ndarray): sigmoid(z), with the same shape as z.
         
    """
    g = 1 / (1 + np.exp(-z))
    
    return g

Then, I checked to see if the sigmoid function was implemented correctly.

In [11]:
def test_sigmoid():
    # Case 1: z is a scalar
    z1 = 0
    expected1 = 0.5
    assert np.isclose(sigmoid(z1), expected1), f"Expected {expected1}, but got {sigmoid(z1)}"

    # Case 2: z is a numpy array of shape (5,)
    z2 = np.array([0, 2, -2, 1, -1])
    expected2 = np.array([0.5, 0.88079708, 0.11920292, 0.73105858, 0.26894142])
    assert np.allclose(sigmoid(z2), expected2), f"Expected {expected2}, but got {sigmoid(z2)}"

    # Case 3: z is a matrix
    z3 = np.array([[0, 2], [-2, 1]])
    expected3 = np.array([[0.5, 0.88079708], [0.11920292, 0.73105858]])
    assert np.allclose(sigmoid(z3), expected3), f"Expected {expected3}, but got {sigmoid(z3)}"

    print("All test cases passed!")

test_sigmoid()

All test cases passed!


So, the sigmoid function works properly.

## 3.2. Cost function

Next, I implemented the cost function. For logistic regression, the cost function is of the form: 
$$ J(\mathbf{w},b) = \frac{1}{m}\sum_{i=0}^{m-1} \left[ loss(f_{\mathbf{w},b}(\mathbf{x}^{(i)}), y^{(i)}) \right]$$

where
* m is the number of training examples in the dataset


* The cost for a single data point is calculated as:

    $$loss(f_{\mathbf{w},b}(\mathbf{x}^{(i)}), y^{(i)}) = (-y^{(i)} \log\left(f_{\mathbf{w},b}\left( \mathbf{x}^{(i)} \right) \right) - \left( 1 - y^{(i)}\right) \log \left( 1 - f_{\mathbf{w},b}\left( \mathbf{x}^{(i)} \right) \right)$$
    
    
*  $f_{\mathbf{w},b}(\mathbf{x}^{(i)})$ is the model's prediction

*  $y^{(i)}$ is the actual label

*  $f_{\mathbf{w},b}(\mathbf{x}^{(i)}) = g(\mathbf{w} \cdot \mathbf{x^{(i)}} + b)$ where function $g$ is the sigmoid function.

In [12]:
def cost_function(X, y, w, b):
    """
    Computes the cost over all examples
    Args:
      X : (ndarray Shape (m,n)) data, m examples by n features
      y : (ndarray Shape (m,))  target value 
      w : (ndarray Shape (n,))  values of parameters of the model      
      b : (scalar)              value of bias parameter of the model
    Returns:
      cost : (scalar) cost 
    """
    m, n = X.shape
    
    # z = X.dot(w) + b
    # g = sigmoid(z)
    # loss = (-y*np.log(g)) - ((1-y)*np.log(1-g))
    # cost = 1/m * np.sum(loss)
    
    cost = 0
    for i in range(m):
        z = np.dot(X[i],w) + b
        f_wb = sigmoid(z)
        cost += -y[i]*np.log(f_wb) - (1-y[i])*np.log(1-f_wb)
    cost = cost/m
    
    return cost

In [13]:
def test_cost_function():
    # Define test cases
    X_test_case = np.array([[1, 2], [3, 4]])
    y_test_case = np.array([0, 1])
    w_test_case = np.array([0.1, 0.2])
    b_test_case = 0.5

    # Expected cost
    expected_cost = 0.749

    # Compute the cost using the cost_function
    computed_cost = cost_function(X_test_case, y_test_case, w_test_case, b_test_case)

    # Check if the computed cost is close to the expected cost
    assert np.isclose(computed_cost, expected_cost, atol=0.01), f"Expected {expected_cost}, but got {computed_cost}"

    print("Cost function test passed!")

test_cost_function()

Cost function test passed!


So, the cost function works fine.

## 3.3. Gradient descent

In this part, I implemented the gradient descent algorithm.

The pseudo code for gradient descent algorithm is:
$$\begin{align*}& \text{repeat until convergence:} \; \lbrace \newline \; & b := b -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial b} \newline       \; & w_j := w_j -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial w_j} \tag{1}  \; & \text{for j := 0..n-1}\newline & \rbrace\end{align*}$$

where, parameters $b$ and $w_j$ are all updated simultaniously

For logistic regression, the gradients are calculated as:
$$
\frac{\partial J(\mathbf{w},b)}{\partial b}  = \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - \mathbf{y}^{(i)}) \tag{2}
$$
$$
\frac{\partial J(\mathbf{w},b)}{\partial w_j}  = \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - \mathbf{y}^{(i)})x_{j}^{(i)} \tag{3}
$$
* m is the number of training examples in the dataset.
*  $f_{\mathbf{w},b}(x^{(i)})$ is the model's prediction.
*  $y^{(i)}$ is the actual label.

First, I created a function for calculating the gradient.

In [22]:
def compute_gradient(X, y, w, b, lambda_=None):
    """
    Computes the gradient for logistic regression 
 
    Args:
      X : (ndarray Shape (m,n)) data, m examples by n features
      y : (ndarray Shape (m,))  target value 
      w : (ndarray Shape (n,))  values of parameters of the model      
      b : (scalar)              value of bias parameter of the model
    Returns
      dj_dw : (ndarray Shape (n,)) The gradient of the cost with respect to the parameters w. 
      dj_db : (scalar)             The gradient of the cost with respect to the parameter b. 
    """
    m, n = X.shape
    dj_dw = np.zeros(w.shape)
    
    z_wb = X.dot(w) + b
    f_wb = sigmoid(z_wb)

    dj_db = np.sum(f_wb - y) / m

    for i in range(n):
        dj_dw[i] = np.sum(np.dot((f_wb - y), (X.T)[i])) / m
   
    return dj_db, dj_dw

In [23]:
def test_compute_gradient():
    # Define test cases
    X_test_case = np.array([[1, 2], [3, 4]])
    y_test_case = np.array([0, 1])
    w_test_case = np.array([0.1, 0.2])
    b_test_case = 0.5

    # Expected gradients
    expected_dj_db = 0.2816
    expected_dj_dw = np.array([0.1135, 0.3951])

    # Compute the gradients using the compute_gradient function
    computed_dj_db, computed_dj_dw = compute_gradient(X_test_case, y_test_case, w_test_case, b_test_case)

    # Check if the computed gradients are close to the expected gradients
    assert np.isclose(computed_dj_db, expected_dj_db, atol=0.01), f"Expected dj_db {expected_dj_db}, but got {computed_dj_db}"
    assert np.allclose(computed_dj_dw, expected_dj_dw, atol=0.01), f"Expected dj_dw {expected_dj_dw}, but got {computed_dj_dw}"

    print("Compute gradient test passed!")

test_compute_gradient()

Compute gradient test passed!


The function for computing gradient works just fine.

Next, I created a function to perform the gradient descent algorithm.

In [24]:
def gradient_descent(X, y, w_in, b_in, cost_function, gradient_function, alpha, num_iters, lambda_): 
    """
    Performs batch gradient descent to learn theta. Updates theta by taking 
    num_iters gradient steps with learning rate alpha
    
    Args:
      X :    (ndarray Shape (m, n) data, m examples by n features
      y :    (ndarray Shape (m,))  target value 
      w_in : (ndarray Shape (n,))  Initial values of parameters of the model
      b_in : (scalar)              Initial value of parameter of the model
      cost_function :              function to compute cost
      gradient_function :          function to compute gradient
      alpha : (float)              Learning rate
      num_iters : (int)            number of iterations to run gradient descent
      lambda_ : (scalar, float)    regularization constant
      
    Returns:
      w : (ndarray Shape (n,)) Updated values of parameters of the model after
          running gradient descent
      b : (scalar)                Updated value of parameter of the model after
          running gradient descent
    """
    
    # number of training examples
    m = len(X)
    
    # An array to store cost J and w's at each iteration primarily for graphing later
    J_history = []
    w_history = []
    
    for i in range(num_iters):

        # Calculate the gradient and update the parameters
        dj_db, dj_dw = gradient_function(X, y, w_in, b_in, lambda_)   

        # Update Parameters using w, b, alpha and gradient
        w_in = w_in - alpha * dj_dw               
        b_in = b_in - alpha * dj_db              
       
        # Save cost J at each iteration
        if i<100000:      # prevent resource exhaustion 
            cost =  cost_function(X, y, w_in, b_in, lambda_)
            J_history.append(cost)

        # Print cost every at intervals 10 times or as many iterations if < 10
        if i% math.ceil(num_iters/10) == 0 or i == (num_iters-1):
            w_history.append(w_in)
            print(f"Iteration {i:4}: Cost {float(J_history[-1]):8.2f}   ")
        
    return w_in, b_in, J_history, w_history #return w and J,w history for graphing

In [25]:
np.random.seed(1)
initial_w = 0.01 * (np.random.rand(30) - 0.5)
initial_b = -8

# Some gradient descent settings
iterations = 1000
alpha = 0.001

w,b, J_history,_ = gradient_descent(X_train ,y_train, initial_w, initial_b, 
                                   cost_function, compute_gradient, alpha, iterations, 0)

TypeError: cost_function() takes 4 positional arguments but 5 were given

In [9]:
def predict(X, w, b): 
    """
    Predict whether the label is 0 or 1 using learned logistic
    regression parameters w
    
    Args:
      X : (ndarray Shape (m,n)) data, m examples by n features
      w : (ndarray Shape (n,))  values of parameters of the model      
      b : (scalar)              value of bias parameter of the model

    Returns:
      p : (ndarray (m,)) The predictions for X using a threshold at 0.5
    """
    # number of training examples
    m, n = X.shape   
    p = np.zeros(m)
    
    # Loop over each example
    for i in range(m):   
        z_wb = np.dot(w, X.T) + b
        
        # Calculate the prediction for this example
        f_wb = sigmoid(z_wb)

        # Apply the threshold
        p[i] = 1 if f_wb[i] >= 0.5 else 0
    return p

In [8]:
def compute_cost_reg(X, y, w, b, lambda_ = 1):
    """
    Computes the cost over all examples
    Args:
      X : (ndarray Shape (m,n)) data, m examples by n features
      y : (ndarray Shape (m,))  target value 
      w : (ndarray Shape (n,))  values of parameters of the model      
      b : (scalar)              value of bias parameter of the model
      lambda_ : (scalar, float) Controls amount of regularization
    Returns:
      total_cost : (scalar)     cost 
    """

    m, n = X.shape
    
    # Calls the compute_cost function that you implemented above
    cost_without_reg = cost_function(X, y, w, b) 
    
    # You need to calculate this value
    reg_cost = 0.
    
    ### START CODE HERE ###
    reg_cost = lambda_ / (2*m) * np.sum(np.square(w))
        
    
    ### END CODE HERE ### 
    
    # Add the regularization cost to get the total cost
    total_cost = cost_without_reg + reg_cost

    return total_cost

In [ ]:
def compute_gradient_reg(X, y, w, b, lambda_ = 1): 
    """
    Computes the gradient for logistic regression with regularization
 
    Args:
      X : (ndarray Shape (m,n)) data, m examples by n features
      y : (ndarray Shape (m,))  target value 
      w : (ndarray Shape (n,))  values of parameters of the model      
      b : (scalar)              value of bias parameter of the model
      lambda_ : (scalar,float)  regularization constant
    Returns
      dj_db : (scalar)             The gradient of the cost w.r.t. the parameter b. 
      dj_dw : (ndarray Shape (n,)) The gradient of the cost w.r.t. the parameters w. 

    """
    m, n = X.shape
    
    dj_db, dj_dw = compute_gradient(X, y, w, b)

    ### START CODE HERE ###     
    regularization = lambda_ / m * w
    dj_dw = dj_dw + regularization
        
    ### END CODE HERE ###         
        
    return dj_db, dj_dw

Comparision

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

# Create an instance of LogisticRegression
log_reg = LogisticRegression()

# Fit the model on the training data
log_reg.fit(X_train, y_train)

log_reg.score(X_test, y_test)


0.956140350877193

In [22]:
y_pred = log_reg.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.96
